In [ ]:
import pandas as pd

melb_df = pd.read_csv('data\melb_data_fe.csv')
#melb_df.head()
display(melb_df.info())

In [4]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)

In [ ]:

# выделяем из даты квартал и согдаем столбец quarter в общей таблице
melb_df['quarter'] = melb_df['Date'].dt.quarter
melb_df.head()


Задание 1.1



In [16]:
# В качестве ответа запишите число объектов, проданных в этом квартале.
melb_df['quarter'].value_counts()

3    4873
2    4359
4    2329
1    2019
Name: quarter, dtype: int64

Задание 1.2

In [1]:
# Преобразуйте все столбцы, в которых меньше 150 уникальных значений, 
# в тип данных category, исключив из преобразования столбцы Date,
# Rooms, Bedroom, Bathroom, Car.# В качестве ответа запишите результирующее
# количество столбцов, которые имеют тип данных category
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())

NameError: name 'melb_df' is not defined

2. Сортировка данных в DataFrame

МЕТОД SORT_VALUES()

In [ ]:
# Отсортируем таблицу по возрастанию цены объектов недвижимости (Price):
melb_df.sort_values(by='Price').head(10)

In [ ]:
# А теперь отсортируем таблицу по убыванию (от самой последней до самой первой) даты продажи объекта (Date)
melb_df.sort_values(by='Date', ascending=False)

СОРТИРОВКА ПО ЗНАЧЕНИЯМ НЕСКОЛЬКИХ СТОЛБЦОВ

In [ ]:
# Так, например, отсортируем таблицу сначала по возрастанию расстояния от центра города (Distance),
# а затем — по возрастанию цены объекта (Price). Для того чтобы вывод был более наглядным, выделим 
# каждую десятую строку из столбцов Distance и Price результирующей таблицы
melb_df.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]

КОМБИНИРОВАНИЕ СОРТИРОВКИ С ФИЛЬТРАЦИЕЙ

In [27]:
# Найдём информацию о таунхаусах (Type), проданных компанией (SellerG) McGrath,
# у которых коэффициент соотношения площадей здания и участка (AreaRatio) меньше -0.8. 
# Результат отсортируем по дате продажи (Date) в порядке возрастания, а после проведём 
# сортировку по убыванию коэффициента соотношения площадей. Также обновим старые индексы на новые
# установив параметр ignore_index на True. Для наглядности результата выберем из таблицы 
# только столбцы Data и AreaRatio:
mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

,Date,AreaRatio
0,2016-07-26,-0.974922
1,2016-09-24,-0.971831
2,2016-11-27,-0.953608
3,2016-12-11,-0.945946
4,2017-08-04,-0.947368
5,2017-08-04,-0.970874


Задание 2.2

In [71]:
# Произведите сортировку столбца AreaRatio по убыванию. При этом индексы полученной таблицы
# замените на новые. Какое значение площади здания находится в строке 1558? Ответ округлите
# до целого числа.
melb_df_copy = melb_df.copy()
melb_df_copy.sort_values(by='AreaRatio', ascending=False, ignore_index=True).loc[1558, ['BuildingArea']]


BuildingArea    126.0
Name: 1558, dtype: object

Задание 2.3


In [84]:
# Найдите таунхаусы (Type) с количеством жилых комнат (Rooms) больше 2.
# Отсортируйте полученную таблицу сначала по возрастанию числа комнат,
# а затем по убыванию средней площади комнат (MeanRoomsSquare).
# Индексы таблицы замените на новые. Какая цена будет у объекта в строке 18?
# Ответ запишите в виде целого числа.
melb_df_copy = melb_df.copy()
#melb_df_copy.info()
mask1 = melb_df_copy['Rooms'] > 2
mask2 = melb_df_copy['Type'] == 'townhouse'
melb_df_copy[mask1 & mask2].sort_values(by=['Rooms', 'MeanRoomsSquare'], ascending=[True, False], ignore_index=True).loc[18, ['Price']]


Price    1300000.0
Name: 18, dtype: object

3. Группировка данных в DataFrame

МЕТОД GROUPBY()

ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С ОДНОЙ АГРЕГАЦИЕЙ

In [ ]:
# Применим агрегирующую функцию среднего к результату работы groupby().
# В качестве столбца для группировки возьмём столбец типа объекта недвижимости (Type):
melb_df.groupby(by='Type').mean()

In [86]:
# Как правило, нам не нужна информация обо всех столбцах,
# поэтому агрегирующие методы можно применять только к интересующему нас столбцу
melb_df.groupby('Type')['Price'].mean()

Type
house        1.242665e+06
townhouse    9.337351e+05
unit         6.051275e+05
Name: Price, dtype: float64

In [87]:
# Теперь давайте выясним, какие регионы (Regionname) наиболее удалены от центра Мельбурна.
# Для этого найдём минимальное значение расстояния от центра города до объекта
# в зависимости от его региона. Результат отсортируем по убыванию расстояния:
melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

Regionname
Western Victoria              29.8
Eastern Victoria              25.2
Northern Victoria             21.8
South-Eastern Metropolitan    14.7
Eastern Metropolitan           7.8
Western Metropolitan           4.3
Southern Metropolitan          0.7
Northern Metropolitan          0.0
Name: Distance, dtype: float64

ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С НЕСКОЛЬКИМИ АГРЕГАЦИЯМИ

In [88]:
# Давайте построим таблицу для анализа продаж по месяцам. Для этого найдём количество продаж,
# а также среднее и максимальное значения цен объектов недвижимости (Price),
# сгруппированных по номеру месяца продажи (MonthSale). Результат отсортируем
# по количеству продаж в порядке убывания:
melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)

,count,mean,max
MonthSale,,,
8,1850,1.056371e+06,6500000.0
7,1835,9.314698e+05,9000000.0
5,1644,1.097807e+06,8000000.0
6,1469,1.068981e+06,7650000.0
3,1408,1.146762e+06,5600000.0
4,1246,1.050479e+06,5500000.0
9,1188,1.126349e+06,6400000.0
10,854,1.135970e+06,6250000.0
11,750,1.142503e+06,5050000.0


In [ ]:
# Более того, метод agg() поддерживает использование и других функций.
# Передадим дополнительно встроенную функцию set, чтобы получить множество
# из агентств недвижимости, которые работают в каждом из регионов:
melb_df.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
)

Задание 3.1

In [101]:
# Сгруппируйте данные по признаку количества комнат и найдите среднюю цену
# объектов недвижимости в каждой группе. В качестве ответа запишите количество комнат, 
# для которых средняя цена наибольшая.
melb_df.groupby('Rooms')['Price'].mean().sort_values(ascending=False)

Rooms
7     1.920700e+06
5     1.870260e+06
6     1.849366e+06
8     1.602750e+06
4     1.445282e+06
3     1.076081e+06
10    9.000000e+05
2     7.750812e+05
1     4.338245e+05
Name: Price, dtype: float64

Задание 3.2

In [ ]:
# Какой регион имеет наименьшее стандартное отклонение
# по географической широте (Lattitude)?
melb_df.groupby('Regionname')['Price'].std().sort_values(ascending=False)
melb_df




Задание 3.3

In [ ]:
# Какая риелторская компания (SellerG) имеет наименьшую общую выручку за период 
# с 1 мая по 1 сентября (включительно) 2017 года?
#Для ответа на этот вопрос рассчитайте сумму продаж (Price) каждой компании в заданный период.
period = melb_df['Date'] >= '2017-05-01'
period_ = '2017-09-01' >= melb_df['Date']
melb_df[period & period_].groupby('SellerG')['Price'].sum().sort_values(ascending=False)


4. Сводные таблицы

МЕТОД GROUPBY КАК СПОСОБ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ

In [ ]:
melb_df.groupby('Rooms')[['Price', 'BuildingArea']].median()
# Series с иерархическими индексами
melb_df.groupby(['Rooms', 'Type'])['Price'].mean()
# переопределить вложенный индекс в виде столбцов таблицы
melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

МЕТОД PIVOT_TABLE ДЛЯ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ

In [ ]:
melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
    fill_value=0
).round()

In [4]:
melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count'
)

Weekend,0,1
Regionname,,
Eastern Metropolitan,447,1024
Eastern Victoria,13,40
Northern Metropolitan,1258,2632
Northern Victoria,11,30
South-Eastern Metropolitan,123,327
Southern Metropolitan,1534,3161
Western Metropolitan,960,1988
Western Victoria,8,24


In [ ]:
# передать в аргумент aggfunc список из агрегирующих функций.
melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

МНОГОМЕРНЫЕ СВОДНЫЕ ТАБЛИЦЫ

In [ ]:
melb_df.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)

ДОСТУП К ДАННЫМ В СВОДНОЙ ТАБЛИЦЕ

In [ ]:
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)
pivot.columns
#  из таблицы pivot мы можем получить средние значения площадей участков 
# для типа здания unit, просто последовательно обратившись по имени столбцов:
display(pivot['mean']['unit'])

In [138]:
# Аналогично производится и фильтрация данных. Например, если нам нужны регионы, 
# в которых средняя площадь здания для домов типа house меньше их медианной площади, 
# то мы можем найти их следующим образом:
mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)


median                       mean              \
Type                   house townhouse unit       house   townhouse   
Regionname                                                            
Southern Metropolitan  586.0     246.0    0  569.643881  278.858824   
Western Metropolitan   531.0     198.0   62  507.883406  244.560669   

                                   
Type                         unit  
Regionname                         
Southern Metropolitan  466.380245  
Western Metropolitan   557.637232

In [139]:
# Чтобы получить индексы отфильтрованной таблицы, можно воспользоваться атрибутом index
# и обернуть результат в список:
print(list(filtered_pivot.index))

['Southern Metropolitan', 'Western Metropolitan']


Задание 4.2

In [2]:
# Составьте сводную таблицу, которая показывает зависимость медианной площади (BuildingArea)
# здания от типа объекта недвижимости (Type) и количества жилых комнат в доме (Rooms).
# Для какой комбинации признаков площадь здания наибольшая?
import numpy as np

melb_df.pivot_table(
    values='BuildingArea',
    index='Rooms',
    columns='Type',
    fill_value=0,
    aggfunc=np.median
).round()

NameError: name 'melb_df' is not defined

Задание 4.3

In [ ]:
# Составьте сводную таблицу, которая показывает зависимость медианной цены объекта
# недвижимости (Price) от риелторского агентства (SellerG) и типа здания (Type).
# Во вновь созданной таблице найдите агентство, у которого медианная цена для зданий 
# типа unit максимальна. 
melb_df_unit_max = melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    fill_value=0,
    aggfunc=np.median
).round()
melb_df_unit_max['unit'].max()

5. Объединение DataFrame: знакомимся с новыми данными

In [34]:
import pandas as pd

data_movies = pd.read_csv('data\movies.csv')
data_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


Задание 5.2

In [ ]:
# Сколько уникальных фильмов представлено в таблице movies?
data_movies

Задание 5.3

In [2]:
# Сколько уникальных пользователей в таблице ratings1?
import pandas as pd
ratings1 = pd.read_csv('data/ratings1.csv')
ratings2 = pd.read_csv('data/ratings2.csv')
dates = pd.read_csv('data/dates.csv')
movies = pd.read_csv('data/movies.csv')


Задание 5.4

In [3]:
# В каком году было выставлено больше всего оценок?
data_dates = pd.read_csv('data/dates.csv')
# приводим к стандарту datetime
data_dates['date'] = pd.to_datetime(data_dates['date'], errors='coerce')
# приводим дату к году
data_dates['date'] = data_dates['date'].dt.year
# находим топ года
data_dates.mode()


,date
0,2000


6. Объединение DataFrame: concat

In [4]:
# ignore_index=True позволяет переделать индексацию а не оставлять старую
ratings = pd.concat([ratings1, ratings2],
    ignore_index=True)
display(ratings)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100832,610,166534,4.0
100833,610,168248,5.0
100834,610,168250,5.0
100835,610,168252,5.0


In [5]:
# в таблицах есть дубль, его удаляем и индексируем заново, метод .drop_duplicates
ratings = ratings.drop_duplicates(ignore_index=True)
print('Число строк в таблице ratings: ', ratings.shape[0])

Число строк в таблице ratings:  100836


In [6]:
# Наконец, мы можем добавить к нашей таблице с оценками даты их выставления.
# Для этого конкатенируем таблицы ratings и dates по столбцам:
ratings_dates = pd.concat([ratings, dates], axis=1)
display(ratings_dates.tail(7))

,userId,movieId,rating,date
100829,610,164179,5.0,2017-05-03 21:07:11
100830,610,166528,4.0,2017-05-04 06:29:25
100831,610,166534,4.0,2017-05-03 21:53:22
100832,610,168248,5.0,2017-05-03 22:21:31
100833,610,168250,5.0,2017-05-08 19:50:47
100834,610,168252,5.0,2017-05-03 21:19:12
100835,610,170875,3.0,2017-05-03 21:20:15


Задание 6.2

In [17]:
# Какой из приведённых ниже способов будет верным при объединении этих таблиц?
df1 = pd.DataFrame({"Name": ["Pankaj", "Lisa"], "Surname": ["Sobolev", "Krasnova"]})
df2 = pd.DataFrame({"Role": ["Admin", "Editor"]})

df = pd.concat([df1, df2], axis=1)
df

,Name,Surname,Role
0,Pankaj,Sobolev,Admin
1,Lisa,Krasnova,Editor


In [23]:
# Список названий всех файлов, находящихся в директории, 
# вы можете получить с помощью функции
print(os.listdir('data/'))

['dates.csv', 'melb_data_fe.csv', 'movies.csv', 'ratings1.csv', 'ratings2.csv']


ЗАДАНИЕ 6.3

In [ ]:
import pandas as pd
import os
print(os.listdir('./Root/users/'))
def concat_users_files(path):
    """
    Вам необходимо написать функцию concat_user_files(path), параметром которой 
    является path - путь до директории.     Функция должна объединить информацию 
    из предоставленных вам файлов в один DataFrame и вернуть его. Не забудьте обновить
    индексы результирующей таблицы после объединения. Учтите тот момент, что в результате
    объединения могут возникнуть дубликаты, от которых необходимо будет избавиться. 
    """
def concat_users_files(path):
    data = pd.DataFrame()
    file_names = os.listdir(path)
    file_names.sort()
    for file in file_names:
        tmp_data = pd.read_csv(path + '/' + file)
        data = pd.concat([data, tmp_data], axis=0, ignore_index=True)
    data = data.drop_duplicates()
    return data

In [44]:
import pandas as pd

def concat_users_files(path):  
#path = 'data/users/'
    list_user = os.listdir(path)
    list_user.sort()
    for i in range(len(list_user)):
        list_user[i] = pd.read_csv(f'{path}{list_user[i]}')
    result = pd.concat(list_user, axis=0, ignore_index=True)
    result = result.drop_duplicates()
    return result
print(concat_users_files('data/users/'))

# list_user[i] = pd.read_csv(f'{path}{list_user[i]}')
#                                                 ^
# SyntaxError: invalid syntax



  user_id                         image_url
0   id001  http://example.com/img/id001.png
1   id002  http://example.com/img/id002.jpg
2   id003  http://example.com/img/id003.bmp
3   id004  http://example.com/img/id004.jpg
4   id005  http://example.com/img/id005.png
5   id006  http://example.com/img/id006.png
6   id007  http://example.com/img/id007.png
7   id008  http://example.com/img/id008.png


7. Объединение DataFrame: join, merge

In [7]:
joined_false = ratings_dates.join(
    movies,
    rsuffix='_right',
    how='left'
)
display(joined_false)

,userId,movieId,rating,date,movieId_right,title,genres
0,1,1,4.0,2000-07-30 18:45:03,1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2.0,Jumanji (1995),Adventure|Children|Fantasy
2,1,6,4.0,2000-07-30 18:37:04,3.0,Grumpier Old Men (1995),Comedy|Romance
3,1,47,5.0,2000-07-30 19:03:35,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance
4,1,50,5.0,2000-07-30 18:48:51,5.0,Father of the Bride Part II (1995),Comedy
...,...,...,...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22,NaN,NaN,NaN
100832,610,168248,5.0,2017-05-03 22:21:31,NaN,NaN,NaN
100833,610,168250,5.0,2017-05-08 19:50:47,NaN,NaN,NaN
100834,610,168252,5.0,2017-05-03 21:19:12,NaN,NaN,NaN


In [ ]:
joined = ratings_dates.join(
    movies.set_index('movieId'),
    on='movieId',
    how='left'
)
display(joined)

МЕТОД ОБЪЕДИНЕНИЯ MERGE

In [10]:
merged = ratings_dates.merge(
    movies,
    on='movieId',
    how='left'
)
display(merged.head())

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [11]:
merge_ratings = ratings1.merge(ratings2, how='outer')
print('Число строк в таблице merge_ratings: ', merge_ratings.shape[0])
display(merge_ratings)

Число строк в таблице merge_ratings:  100836


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


Задание 7.4

In [20]:
a = pd.DataFrame({'A': ['a', 'b', 'c'], 'B': [103, 214, 124], 'C': [1, 4, 2]})
b = pd.DataFrame({'V': ['d', 'b', 'c'], 'U': [1393.7, 9382.2, 1904.5], 'C': [1, 3, 2]})
c = a.merge(b, how='right', on='C')
c

,A,B,C,V,U
0,a,103.0,1,d,1393.7
1,NaN,NaN,3,b,9382.2
2,c,124.0,2,c,1904.5


ЗАДАНИЕ 7.5

In [38]:
items_df = pd.DataFrame({
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 
    'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
    'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
})
purchase_df = pd.DataFrame({
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
})
#print(items_df, '\n', purchase_df )
merged = items_df.merge(purchase_df, how='inner')
income = sum(merged['stock_count'] * merged['price'])
income

19729490

8. Закрепление знаний

In [1]:
import re 
# Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
# У скольких фильмов не указан год их выпуска?

def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None
    
joined['year_release'] = joined['title'].apply(get_year_release)
uu = joined['date'].count() - joined['year_release'].count()
uu

NameError: name 'joined' is not defined

Задание 8.2

In [ ]:

# Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
mask = joined['year_release'] == 1999
mask2 = joined[mask].groupby(by='title')['rating'].mean().sort_values(ascending=False)
mask2

Задание 8.3

In [107]:
# Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
mask = joined['year_release'] == 2010
mask2 = joined[mask].groupby(by='genres')['rating'].mean().sort_values(ascending=False)
mask2

genres
Animation|Drama|Fantasy|Mystery      5.000000
Animation|Children|Mystery           5.000000
Adventure|Children|Comedy|Mystery    5.000000
Comedy|Musical                       5.000000
Sci-Fi                               4.750000
                                       ...   
Adventure|Comedy|Fantasy             1.833333
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Action|Sci-Fi                        1.000000
Name: rating, Length: 119, dtype: float64

Задание 8.4

In [ ]:
# Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) комбинаций жанров
# (genres) фильмов? В качестве ответа запишите идентификатор этого пользователя.
#mask2 = joined.groupby(by='userId')['genres'].u.sort_values(ascending=False)
joined.groupby('userId')['genres'].agg(
    		['nunique', set]
).sort_values(by='nunique', ascending=False)

Задание 8.5

In [ ]:
joined

In [ ]:
# Найдите пользователя, который выставил наименьшее количество оценок, 
# но его средняя оценка фильмам наибольшая.
joined.groupby('userId')['rating'].agg(
    ['count', 'mean']
).sort_values(by='count', ascending=False)

Задание 8.6

In [126]:
# Найдите сочетание жанров (genres) за 2018 году, которое имеет наибольший средний рейтинг 
# (среднее по столбцу rating), и при этом число выставленных ему оценок 
# (количество значений в столбце rating) больше 10.
mask = joined['year_release'] == 2018
res_1 = joined[mask].groupby(by='genres')['rating'].agg(
    ['count', 'mean']
).sort_values(by='count')
mask3 = res_1['count'] > 10
res_1[mask3]['mean'].sort_values()


genres
Action|Comedy|Sci-Fi       3.875000
Action|Adventure|Sci-Fi    3.928571
Name: mean, dtype: float64

Задание 8.7

In [154]:
# Добавьте в таблицу новый признак year_rating — год выставления оценки. 
# Создайте сводную таблицу, которая иллюстрирует зависимость среднего рейтинга фильма 
# от года выставления оценки и жанра. Выберите верные варианты ответа, исходя из построенной таблицы:
joined['date'] = pd.to_datetime(joined['date'], dayfirst=True)
joined['year_rating'] = joined['date'].dt.year

# За весь период (с 1996 по 2018 год) сочетание жанров Action|Adventure ни разу не 
# получало среднюю оценку ниже 3.
mask = joined['year_release'] > 1996
mask2 = joined['year_release'] <= 2018 
mask3 = joined['genres'] == 'Action|Adventure'
yy = joined[mask & mask2 & mask3].groupby(by='genres')['rating'].mean().sort_values(ascending=False)
# Неверно Action|Adventure    2.852713

# Наилучшую оценку жанр Action|Adventure|Animation|Children|Comedy|IMAX получил в 2010 году.
mask = joined['year_release'] == 2010
mask3 = joined['genres'] == 'Action|Adventure|Animation|Children|Comedy|IMAX'
joined[mask & mask3].groupby(by='genres')['rating'].count()
joined[mask3]['rating'].count()




genres
Action|Adventure    2.852713
Name: rating, dtype: float64

Задание 8.8

In [ ]:
import pandas as pd

orders = pd.read_csv('data/orders.csv', sep=';')
products = pd.read_csv('data/products.csv', sep=';')
orders.rename(columns={'ID товара': 'Product_ID'}, inplace=True)

# Какой идентификатор (Order ID) имеет заказ, для которого не оказалось информации о товаре?
merged = orders.merge(
    products,
    on='Product_ID',
    how='left'
)
merged


Задание 8.10

In [43]:
# Какой покупатель принёс наибольшую суммарную прибыль интернет-магазину за указанный период?
# В ответ запишите идентификатор этого покупателя (ID Покупателя).
# Прибыль состоит только из оплаченных заказов и рассчитывается как количество купленного товара,
# умноженное на его цену.

merged['Прибыль'] = merged['Price'] * merged['Количество']
#merged['Прибыль_'] = merged['Price'] * merged['Количество']
#merged[mask].groupby(by='ID Покупателя')[mask].sort_values(ascending=False)
merged.groupby(['ID Покупателя'])['Прибыль'].sum().sort_values(ascending=False)

#melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()
#"""for i in range(len(tr['ID Покупателя'])):sum(tr[i]) """"


ID Покупателя
7     17096.0
5     14042.0
9      7040.0
8      3087.0
1      2240.0
10      995.0
Name: Прибыль, dtype: float64